# Computational Theory

## Introduction

>Things to reference later
>
> https://www.markdownguide.org/ for markdown guidance
>

## Imports

In [32]:
import numpy as np

## Problem 1 - Binary Words & Operations

### Implementing Parity

**Purpose**: The Parity function computes the bitwise XOR of 3 unsigned 32-bit integers.  
Each bit in the **output** is set to 1, if there is an odd number of **input** bits as 1; Otherwise 0


**Spec.** Defined in the Secure Hash Standard (FIPS 180-4, p.10) as:
>
>Parity(x, y, z) = x XOR y XOR z
>
See: [FIPS 180-4](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)


**Intuition**: Fairly simple function  
Just flips the bits based off of odd number of inputs as above.
- If 1 or 3 of them are 1, Output -> 1
- If 0 or 2 of them are 1, Output -> 0


**Example**  
X       =   0110  
Y       =   1110  
Z       =   0010  

Output  =   1010

In [33]:
def parity(x, y, z):
    """
    Calculate the parity (bitwise XOR) of 3 32-bit unsigned integers.

    Parameters
    ----------
        x, y, z (int / numpy.uint32)
            32-bit unsigned integer.

    Returns
    -------
        numpy.uint32
            The parity of the three integers.

    Raises
    ------
        TypeError
            If any argument is not an integer.
    """

    # Loop over each passed parameter
    for args in (x, y, z):
        # Use isInstance to ensure correct type (object, desired type)
        # See: https://www.w3schools.com/python/ref_func_isinstance.asp
        if not isinstance(args, (int, np.integer)):
            # Raises / Throws a TypeError with a message
            raise TypeError("All arguments must be integers. No other types are allowed.")

    # Cast all inputs as NumPy unsigned 32-bit integers.
    # See: https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.uint32
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)

    # Apply bitwise XOR (^) to the three inputs and return
    return x ^ y ^ z

#### Test Cases

In [34]:
def test_valid_parity():
    assert parity(1, 0, 0) == np.uint32(1)
    assert parity(1, 1, 0) == np.uint32(0)
    assert parity(5, 12, 6) == np.uint32(15)

def test_parity_type_error():
    try:
        parity(1.5, 2, 3)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")
    
test_valid_parity()
test_parity_type_error()

print("All test cases pass")

All test cases pass


### Implementing Choose (Ch)

In [35]:
def ch(x, y, z):
    """
    Implementing the choose function used in SHA-256.

    The choose function is defined as:
        Ch(x, y, z) = (x AND y) XOR (NOT x AND z)

    In my own words,
        X acts as a 'mask' , each bit of the 32-bit int is checked
        If the current bit is 1, the i'th bit of y is chosen
        else the i'th bit of z is chosen.
    
    Example:
        x = 100'200'123 =       00000101 11111000 11101110 10111011
        y = 123'200'100 =       00000111 01010111 11100010 01100100
        z = 200'123'100 =       00001011 11101101 10100010 11011100
        Output - If X's current bit is 1 => "Choose" Y's bit
                    Else "Choose" Z's bit
                                00001111 01010101 11100010 01100100

    Args:
        x, y, z (int): 32-bit unsigned integers.
        
    Returns:
        int: The result of the choose function as a 32-bit unsigned integer.
    
    References:
        Used external site for grabbing example values
        See: https://www.binaryconvert.com/convert_unsigned_int.html
    """

    for args in (x, y, z):
        if not isinstance(args, (int, np.integer)):
            raise TypeError("All arguments must be integers. No other types are allowed.")

    # Using the tilde(~) operator to compute the bitwise NOT
    # np.uint32 is used to force the result as a 32-bit unsigned integer
    return np.uint32((x & y) ^ (~x & z))

#### Test Cases

In [36]:
def test_valid_ch():
    assert ch(1, 0, 0) == np.uint32(0)
    assert ch(1, 1, 0) == np.uint32(1)
    assert ch(5, 12, 6) == np.uint32(6)

def test_ch_type_error():
    try:
        ch(1.5, 2, 3)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")

test_valid_ch()
test_ch_type_error()

print("All test cases pass")

All test cases pass


### Implementing Majority (Maj)

In [37]:
def maj(x, y, z):
    """
    Implementing the majority function used in SHA-256.

    The majority function is defined as:
        Maj(x, y, z) = (x AND y) XOR (x AND z) XOR (y AND z)
    
    In my own words,
        The majority function checks each bit of each of the passed ints.
        If 2 or more bits are 1, the output is 1, else 0.
    
    Example:
        X       =    01101001
        Y       =    11010100
        Z       =    11110000
        Output  =    11110000

    Args:
        x, y, z (int): 32-bit unsigned integers.
    
    Returns:
        int: Returns the majority bitwise as 32-bit unsigned integer.
    
    References:
    
    """

    for args in (x, y, z):
        if not isinstance(args, (int, np.integer)):
            raise TypeError("All arguments must be integers. No other types are allowed.")
    
    return np.uint32((x & y) ^ (x & z) ^ (y & z))

#### Test Cases

In [38]:
def test_maj_valid():
    assert maj(1, 0, 0) == np.uint32(0)
    assert maj(1, 1, 0) == np.uint32(1)
    assert maj(5, 12, 6) == np.uint32(4)

def test_maj_type_error():
    try:
        maj(1.5, 2, 3)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")

test_maj_valid()
test_maj_type_error()

print("All test cases pass")


All test cases pass


### Implementing Big & Small Sigmas + Helper Functions

#### Helper Functions
- ROTR -> For rotating bits, shifts bits to the right, if it falls off, move it to the left(start)
- SHR -> Logical shift bits, shifts bits to the right, fill the left with zeros

In [39]:
def rotr(x, n):
    """
    Perform a right rotation on a 32-bit unsigned integer.

    Args:
        x (int): 32-bit unsigned integer to be rotated.
        n (int): Number of positions to rotate.

    Returns:
        int: The result of the right rotation as a 32-bit unsigned integer.
    """
    for args in (x, n):
        if not isinstance(args, (int, np.integer)):
            raise TypeError("All arguments must be integers. No other types are allowed.")

    n = n % 32  # Ensure n is within the range of 0-31
    x = np.uint32(x)  # Cast x before shifting stop overflow error

    if n == 0:
        return x

    return np.uint32((x >> n) | (x << (32 - n)))

In [40]:
def shr(x, n):
    """
    Perform a logical right shift on a 32-bit unsigned integer.

    Args:
        x (int): 32-bit unsigned integer to be shifted.
        n (int): Number of positions to shift.

    Returns:
        int: The result of the logical right shift as a 32-bit unsigned integer.
    """
    for args in (x, n):
        if not isinstance(args, (int, np.integer)):
            raise TypeError("All arguments must be integers. No other types are allowed.")

    return np.uint32(x) >> n # np.uint32 fills left with zeros built in

##### Test Cases (Helper Functions)

In [41]:
def test_rotr_valid():
    assert rotr(10, 1) == np.uint32(5)
    assert rotr(10, 2) == np.uint32(2147483650)
    assert rotr(16, 4) == np.uint32(1)

def test_shr_valid():
    assert shr(10, 1) == np.uint32(5)
    assert shr(10, 2) == np.uint32(2)

def test_rotr_type_error():
    try:
        rotr(1.5, 2)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")

def test_shr_type_error():
    try:
        shr(1.5, 2)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")

test_rotr_valid()
test_shr_valid()
test_rotr_type_error()
test_shr_type_error()

print("All test cases pass")

All test cases pass


#### Big Sigma 0 & 1
- Sigma 0 -> XOR Bitwise with 3 rotations (2, 13, 22) - SHA-256 Reference required here
- Sigma 1 -> XOR Bitwise with 3 rotations (6, 11, 25) - SHA-256 Reference required here

In [42]:
def big_sigma_0(x):
    """
    Implementing the Big Sigma 0 function used in SHA-256.

    The Big Sigma 0 function is defined as:
        Σ0(x) = ROTR^2(x) XOR ROTR^13(x) XOR ROTR^22(x)
    
    Args:
        x (int): 32-bit unsigned integer.
    Returns:
        int: Result of the Big Sigma 0 function as a 32-bit unsigned integer.
    """
    if not isinstance(x, (int, np.integer)):
        raise TypeError("Argument must be an integer. No other types are allowed.")

    x = np.uint32(x)
    return np.uint32(rotr(x, 2) ^ rotr(x, 13) ^ rotr(x, 22))

In [43]:
def big_sigma_1(x):
    """
    Implementing the Big Sigma 1 function used in SHA-256.

    The Big Sigma 1 function is defined as:
        Σ1(x) = ROTR^6(x) XOR ROTR^11(x) XOR ROTR^25(x)
        
    Args:
        x (int): 32-bit unsigned integer.
    Returns:
        int: Result of the Big Sigma 1 function as a 32-bit unsigned integer.
    """
    if not isinstance(x, (int, np.integer)):
        raise TypeError("Argument must be an integer. No other types are allowed.")

    x = np.uint32(x)
    return np.uint32(rotr(x, 6) ^ rotr(x, 11) ^ rotr(x, 25))

##### Test Cases (Big Sigma(s))

In [44]:
def test_bsigma_0_valid():
    assert big_sigma_0(1)  == np.uint32(rotr(1, 2) ^ rotr(1, 13) ^ rotr(1, 22))
    assert big_sigma_0(5)  == np.uint32(rotr(5, 2) ^ rotr(5, 13) ^ rotr(5, 22))
    assert big_sigma_0(10) == np.uint32(rotr(10, 2) ^ rotr(10, 13) ^ rotr(10, 22))

def test_bsigma_1_valid():
    assert big_sigma_1(1)  == np.uint32(rotr(1, 6) ^ rotr(1, 11) ^ rotr(1, 25))
    assert big_sigma_1(5)  == np.uint32(rotr(5, 6) ^ rotr(5, 11) ^ rotr(5, 25))
    assert big_sigma_1(10) == np.uint32(rotr(10, 6) ^ rotr(10, 11) ^ rotr(10, 25))


def test_bsigma_type_error():
    try:
        big_sigma_0(1.5)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")
    
    try:
        big_sigma_1(1.5)
    except TypeError:
        pass
    else:
         raise AssertionError("Expected TypeError for float input")

test_bsigma_0_valid()
test_bsigma_1_valid()
test_bsigma_type_error()

print("All test cases pass")

All test cases pass


#### Small Sigma 0 & 1
- sigma 0 -> XOR Bitwise, 2 rotations (7, 18) & Bitwise shift right (3) - SHA-256 Reference required here
- sigma 1 -> XOR Bitwise, 2 rotations (17, 19) & Bitwise shift right (10) - SHA-256 Reference required here

In [45]:
def small_sigma_0(x):
    """
    Implementing the Small Sigma 0 function used in SHA-256.

    Args:
        x (int): 32-bit unsigned integer.
    
    Returns:
        int: Result of the Small Sigma 0 function as a 32-bit unsigned integer.
    """

    if not isinstance(x, (int, np.integer)):
        raise TypeError("Argument must be an integer. No other types are allowed.")

    x = np.uint32(x)
    return np.uint32(rotr(x, 7) ^ rotr(x, 18) ^ shr(x, 3))

In [46]:
def small_sigma_1(x):
    """
    Implementing the Small Sigma 1 function used in SHA-256.

    Args:
        x (int): 32-bit unsigned integer.
    
    Returns:
        int: Result of the Small Sigma 1 function as a 32-bit unsigned integer.
    """

    if not isinstance(x, (int, np.integer)):
        raise TypeError("Argument must be an integer. No other types are allowed.")

    x = np.uint32(x)
    return np.uint32(rotr(x, 17) ^ rotr(x, 19) ^ shr(x, 10))

##### Test Cases (Small Sigma(s))

In [47]:
def test_ssigma_0_valid():
    assert small_sigma_0(1)  == np.uint32(rotr(1, 7) ^ rotr(1, 18) ^ shr(1, 3))
    assert small_sigma_0(5)  == np.uint32(rotr(5, 7) ^ rotr(5, 18) ^ shr(5, 3))
    assert small_sigma_0(10) == np.uint32(rotr(10, 7) ^ rotr(10, 18) ^ shr(10, 3))

def test_ssigma_type_error():
    try:
        small_sigma_0(1.5)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")
    
    try:
        small_sigma_1(1.5)
    except TypeError:
        pass
    else:
         raise AssertionError("Expected TypeError for float input")

test_ssigma_0_valid()
test_ssigma_type_error()

print("All test cases pass")

All test cases pass


## Problem 2 - Fractional Parts of Cube Roots

## Problem 3 - Padding

## Problem 4 - Hashes

## Problem 5 - Passwords

## Conclusion

## End